In [2]:
import time
import pandas as pd
import numpy as np
import os
from datetime import date
import time
import json
today = date.today()
path = os.getcwd()
print(f'💚 Today is {today}')

💚 Today is 2025-10-29


In [12]:
df = pd.read_csv(path + '/data_abstracts/true_mobility_studies_617.csv')
df_rq = df[(df['rq_cat'].isin([1, 2, 4])) & (df['is_oa'] == True)]
len(df), len(df_rq)

(617, 255)

In [20]:
print(df_rq['doi'].isna().sum())
print(df_rq['landing_page'].isna().sum())
print(df_rq.oa_link.isna().sum())

2
0
1


In [21]:
df_rq[['id', 'doi', 'landing_page', 'oa_link']].to_csv(path + '/data_abstracts/paper_links_255.csv', index=False)

# Download OA papers
* 166 papers worked directly with the oa_link

In [22]:
import requests

# Folder to save PDFs
save_folder = f'{path}/paper_extraction'
os.makedirs(save_folder, exist_ok=True)

for index, row in df_rq.iterrows():
    pdf_url = row['oa_link']
    id_long = row['id']
    pdf_id = id_long.split('https://openalex.org/')[1]
    
    try:
        response = requests.get(pdf_url, timeout=15)
        if response.status_code == 200 and 'application/pdf' in response.headers.get('Content-Type', ''):
            pdf_path = os.path.join(save_folder, f"{pdf_id}.pdf")
            with open(pdf_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {pdf_id}")
        else:
            print(f"Failed to download {pdf_id}: Not a PDF or bad response")
    except Exception as e:
        print(f"Error downloading {pdf_id}: {e}")


Downloaded: W2564827629
Failed to download W1966505070: Not a PDF or bad response
Downloaded: W3041900243
Failed to download W2508091509: Not a PDF or bad response
Downloaded: W4393244952
Downloaded: W3208235548
Downloaded: W3122728969
Downloaded: W3166017557
Downloaded: W4289222991
Downloaded: W2164455384
Downloaded: W2470832101
Downloaded: W2123210148
Failed to download W2802089178: Not a PDF or bad response
Failed to download W2079089263: Not a PDF or bad response
Failed to download W2042572864: Not a PDF or bad response
Downloaded: W2912008487
Failed to download W4407738440: Not a PDF or bad response
Failed to download W3143516863: Not a PDF or bad response
Failed to download W4234924712: Not a PDF or bad response
Downloaded: W3123290206
Downloaded: W3027769937
Failed to download W4295122648: Not a PDF or bad response
Downloaded: W2159878756
Downloaded: W2056728055
Downloaded: W4391164542
Failed to download W4381889820: Not a PDF or bad response
Error downloading W4242326182: Inval

# Coherence Check: LLM as a judge

In [34]:
from openai import OpenAI
from dotenv import load_dotenv
# OpenAI
load_dotenv() 
openai_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=openai_API_KEY)
gpt_model = "o3-mini" #o3-mini-2025-01-31

## RQs

In [39]:
files = os.listdir(path + '/results')
RQ_files = []
for file in files:
    if file.endswith('.csv'):
        if file.startswith('categories_RQ_'):
            RQ_files += [file]

In [83]:
import logging

cat = {}

for idx, file in enumerate(RQ_files):
    print(f'Working .... {file}')
    text_RQ = pd.read_csv(path + '/results/' + file)
    list_RQ = text_RQ['category'].values.tolist()
    formatted_list_RQ = json.dumps(list_RQ, indent=2)
    start_time = time.time()
    completion = openai_client.beta.chat.completions.parse(
            model= gpt_model,
            messages=[
                { 
                "role": "assistant",
                "content": "You are a research assistant in the Social Sciences domain."
                
                },
                {
                    "role": "user", 
                    "content":f"""
                            This is a list of categories of research questions (RQ).
                            ```json
                            {formatted_list_RQ}
                            
                            Compare these categories to the predefined list below and recognize similarities. 
                            Return a JSON object where each key is the predefined category, and the value is a list of similar original categories. 
                            If no similarity is found, skip that category.

                            Predefined categories:
                            cat_1.	Measurement and Methodological Advances
                            cat_2.	Empirical Estimates and Determinants
                            cat_3.	Policy, Institutional, and Geographic Impacts
                            cat_4.	Intergenerational Wealth Mobility and Inheritance
                            cat_5.	Demographic Differences in Mobility (Race, Gender, etc.)
                            cat_6.	Mobility and Non-Income Outcomes (Health, Wellbeing, etc.)
                            cat_7.	Theoretical and Structural Models
                            cat_8.	Perceptions of Mobility and Attitudes
                            cat_9.	Others
                            """
                }
            ],
            response_format={"type": "json_object"}
        )
    results = completion.choices[0].message.content
    cat[idx] = results   
        
    logging.info(f'✅ Done! The process time is: {time.time() - start_time:.2f} seconds')

Working .... categories_RQ_10.csv
Working .... categories_RQ_6.csv
Working .... categories_RQ_7.csv
Working .... categories_RQ_5.csv
Working .... categories_RQ_4.csv
Working .... categories_RQ_3.csv
Working .... categories_RQ_2.csv
Working .... categories_RQ_9.csv
Working .... categories_RQ_8.csv


## Output Management

In [95]:
cat_df = pd.DataFrame(cat.items(), columns=['id', 'result'])
expanded_rows = []

# Iterate through each row
for idx, row in cat_df.iterrows():
    json_string = row['result']
    try:
        parsed = json.loads(json_string)
        for key, values in parsed.items():
            expanded_rows.append({
                'id': row['id'],   # Keep track of original row
                'predefined_category': key,
                'matched_categories': values
            })
    except json.JSONDecodeError:
        print(f"⚠️ JSON parsing error at row {idx}")

# Create expanded DataFrame
df_expanded = pd.DataFrame(expanded_rows)

In [96]:
df_expanded

,id,predefined_category,matched_categories
0,0,cat_1. Regression‐based Measures,[Income Elasticity Measures]
1,0,cat_2. Rank‐based Measures,[Rank‐Based Measures]
2,0,cat_3. Transition Matrix / Probability Measures,[Transition Matrix Measures]
3,0,cat_6. Decomposition / Structural Approaches,[Structural/Decomposition Model Estimates]
4,0,cat_8. Others,[Others]
5,1,cat_1. Regression‐based Measures,[Intergenerational Income Elasticity]
6,1,cat_2. Rank‐based Measures,[Rank‐Based Measures]
7,1,cat_3. Transition Matrix / Probability Measures,[Transition Matrices]
8,1,cat_4. Absolute Mobility Measures,[Absolute Mobility Rates]
9,1,cat_8. Others,[Others]


In [97]:
df_expanded.to_csv(path + '/results/categories_method_LLM_judged.csv', index=False)

# For RQ output:
# id ... file_name
# 0 .... categories_RQ_10.csv
# 1 .... categories_RQ_6.csv
# 2 .... categories_RQ_7.csv
# 3 .... categories_RQ_5.csv
# 4 .... categories_RQ_4.csv
# 5 .... categories_RQ_3.csv
# 6 .... categories_RQ_2.csv
# 7 .... categories_RQ_9.csv
# 8 .... categories_RQ_8.csv



## Data Types

In [82]:
files = os.listdir(path + '/results')
data_files = []
for file in files:
    if file.endswith('.csv'):
        if file.startswith('categories_data_'):
            data_files += [file]

In [88]:
import logging

cat = {}

for idx, file in enumerate(data_files):
    print(f'Working .... {file}')
    text_data = pd.read_csv(path + '/results/' + file)
    list_data = text_data['category'].values.tolist()
    formatted_list_data = json.dumps(list_data, indent=2)
    start_time = time.time()
    completion = openai_client.beta.chat.completions.parse(
            model= gpt_model,
            messages=[
                { 
                "role": "assistant",
                "content": "You are a research assistant in the Social Sciences domain."
                
                },
                {
                    "role": "user", 
                    "content":f"""
                            This is a list of categories to distinguish types of datasets used for a paper.
                            ```json
                            {formatted_list_data}
                            
                            Compare these categories to the predefined list below and recognize similarities. 
                            Return a JSON object where each key is the predefined category, and the value is a list of similar original categories. 
                            If no similarity is found, skip that category.

                            Predefined categories:
                            cat_1.	Panel/Longitudinal Surveys
                            cat_2.	Administrative/Registry Data
                            cat_3.	National Survey Data
                            cat_4.	Opportunity Atlas
                            cat_5.	Natural/Experimental Data
                            cat_6.	Linked Administrative Data
                            cat_7.	International Panel Data
                            cat_8.	Rich List Data
                            cat_9.	University/Institution Data
                            cat_10.	Pseudo-Panel/Household Budget Survey
                            cat_11.	Archival/Historical Data
                            cat_12.	Big Data
                            cat_13.	No dataset
                            cat_14.	Others
                            """
                }
            ],
            response_format={"type": "json_object"}
        )
    results = completion.choices[0].message.content
    cat[idx] = results   
        
    logging.info(f'✅ Done! The process time is: {time.time() - start_time:.2f} seconds')

Working .... categories_data_10.csv
Working .... categories_data_9.csv
Working .... categories_data_8.csv
Working .... categories_data_3.csv
Working .... categories_data_2.csv
Working .... categories_data_6.csv
Working .... categories_data_7.csv
Working .... categories_data_5.csv
Working .... categories_data_4.csv


## Methods

In [92]:
files = os.listdir(path + '/results')
method_files = []
for file in files:
    if file.endswith('.csv'):
        if file.startswith('categories_method_'):
            method_files += [file]

In [94]:
import logging

cat = {}

for idx, file in enumerate(method_files):
    print(f'Working .... {file}')
    text_method = pd.read_csv(path + '/results/' + file)
    list_method = text_method['category'].values.tolist()
    formatted_list_method = json.dumps(list_method, indent=2)
    start_time = time.time()
    completion = openai_client.beta.chat.completions.parse(
            model= gpt_model,
            messages=[
                { 
                "role": "assistant",
                "content": "You are a research assistant in the Social Sciences domain."
                
                },
                {
                    "role": "user", 
                    "content":f"""
                            This is a list of categories to distinguish methods used for a paper.
                            ```json
                            {formatted_list_method}
                            
                            Compare these categories to the predefined list below and recognize similarities. 
                            Return a JSON object where each key is the predefined category, and the value is a list of similar original categories. 
                            If no similarity is found, skip that category.

                            Predefined categories:
                            cat_1. Regression‐based Measures 
                            cat_2. Rank‐based Measures
                            cat_3. Transition Matrix / Probability Measures
                            cat_4. Absolute Mobility Measures
                            cat_5. Multigenerational Measures
                            cat_6. Decomposition / Structural Approaches
                            cat_7. Non‐parametric Approaches
                            cat_8. Others
                            """
                }
            ],
            response_format={"type": "json_object"}
        )
    results = completion.choices[0].message.content
    cat[idx] = results   
        
    logging.info(f'✅ Done! The process time is: {time.time() - start_time:.2f} seconds')

Working .... categories_method_8.csv
Working .... categories_method_9.csv
Working .... categories_method_4.csv
Working .... categories_method_5.csv
Working .... categories_method_7.csv
Working .... categories_method_6.csv
Working .... categories_method_2.csv
Working .... categories_method_10.csv
Working .... categories_method_3.csv


In [3]:
data = {}
measure = {}
rq = {}
for file in os.listdir(path + '/results/coherence-catgen/'):
    if file.endswith('.csv'):
        if file.startswith('categories_data_'):
            data[file] = pd.read_csv(path + '/results/coherence-catgen/' + file)
        if file.startswith('categories_method_'):
            measure[file] = pd.read_csv(path + '/results/coherence-catgen/' + file)
        if file.startswith('categories_RQ_'):
            rq[file] = pd.read_csv(path + '/results/coherence-catgen/' + file)

len(data), len(measure), len(rq)

(9, 9, 9)

{'categories_data_10.csv':                         category  \
 0           US Panel Survey Data   
 1         US Administrative Data   
 2  European Register/Survey Data   
 3                Chinese Surveys   
 4    Other Country-Specific Data   
 5  Global/International Datasets   
 6                     No dataset   
 7                         Others   
 
                                                  ids  
 0  ['853', '862', '869', '870', '873', '876', '87...  
 1                              ['872', '877', '868']  
 2         ['849', '875', '881', '863', '882', '889']  
 3                              ['860', '864', '878']  
 4  {'Brazil': ['847', '884'], 'Taiwan': ['848'], ...  
 5                                            ['856']  
 6                                                 []  
 7                                                 []  ,
 'categories_data_9.csv':                           category  \
 0              Administrative Data   
 1                Survey/Panel 

In [ ]:
measure_df = {
    "Category": [
        "Regression‐based Measures",
        "Rank‐based Measures",
        "Transition Matrix / Probability Measures",
        "Absolute Mobility Measures",
        "Multigenerational Measures",
        "Decomposition / Structural Approaches",
        "Non‐parametric Approaches",
        "Others",
        "N/A"
    ],
    "Manual": [9, 9, 9, 5, 0, 5, 1, 11, 7],
    "LLM":    [9, 9, 9, 5, 2, 5, 2, 10, 1]
}

data_df = {
    "Category": [
        "Panel/Longitudinal Surveys",
        "Administrative/Registry Data",
        "National Survey Data",
        "Opportunity Atlas",
        "Natural/Experimental Data",
        "Linked Administrative Data",
        "International Panel Data",
        "Rich List Data",
        "University/Institution Data",
        "Pseudo-Panel/Household Budget Survey",
        "Archival/Historical Data",
        "Big Data",
        "No dataset",
        "Others",
        "N/A"
    ],
    "Manual": [8, 8, 6, 0, 2, 2, 0, 2, 0, 2, 2, 3, 7, 14, 0],
    "LLM":    [9, 10, 9, 1, 2, 3, 2, 2, 0, 3, 2, 3, 9, 12, 0]
}

rq_df = {
    "Category": [
        "Measurement and Methodological Advances",
        "Empirical Estimates and Determinants",
        "Policy, Institutional, and Geographic Impacts",
        "Intergenerational Wealth Mobility and Inheritance",
        "Demographic Differences in Mobility (Race, Gender, etc)",
        "Mobility and Non-Income Outcomes (Health, Wellbeing, etc)",
        "Theoretical and Structural Models",
        "Perceptions of Mobility and Attitudes",
        "Others",
        "N/A"
    ],
    "Manual": [6, 5, 15, 1, 6, 5, 1, 0, 14, 3],
    "LLM":    [11, 19, 26, 12, 10, 10, 5, 3, 9, 0]
}

measure_df = pd.DataFrame(measure_df)
data_df = pd.DataFrame(data_df)
rq_df = pd.DataFrame(rq_df)


In [15]:
from scipy.stats import chi2_contingency
import pandas as pd

def chi_square_test(df, name):
    df = df.copy()
    # Drop categories with no counts in either group
    df = df[(df["Manual"] + df["LLM"]) > 0]
    
    # Run chi-square test
    chi2, p, dof, expected = chi2_contingency([df["Manual"], df["LLM"]])
    print(f"{name}:")
    print(f"  Chi2 = {chi2:.3f}, p = {p:.4f}, dof = {dof}")
    return chi2, p

# Run for each table
for name, d in [("Measure", measure_df), ("Data", data_df), ("RQ", rq_df)]:
    chi_square_test(d, name)


Measure:
  Chi2 = 6.742, p = 0.5647, dof = 8
Data:
  Chi2 = 3.731, p = 0.9878, dof = 12
RQ:
  Chi2 = 15.928, p = 0.0434, dof = 8
